<a href="https://colab.research.google.com/github/robgon-art/ai-tunes/blob/main/Copy_of_Using_GPT_J_to_Create_a_Band_Name_and_Song_Title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using GPT-J to Create a Band Name and Song Title

This notebook demonstrates how to run the [GPT-J-6B model](https://github.com/kingoflolz/mesh-transformer-jax/#GPT-J-6B). See the link for more details about the model, including evaluation metrics and credits.

Based on [GPT-J-6B Inference Demo.ipynb](https://colab.research.google.com/github/kingoflolz/mesh-transformer-jax/blob/master/colab_demo.ipynb) by [@kingoflolz](https://github.com/kingoflolz)

Tip-o-the-cap to [@johndpope](https://github.com/johndpope) for pointing me to GPT-J

## Install Dependencies

First we download the model and install some dependencies. This step takes at least 5 minutes (possibly longer depending on server load).

!!! **Make sure you are using a TPU runtime!** !!!

In [ ]:
!apt install zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

!time tar -I zstd -xf step_383500_slim.tar.zstd

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12 tensorflow==2.5.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 278 kB of archives.
After this operation, 1,141 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 zstd amd64 1.3.3+dfsg-2ubuntu1.2 [278 kB]
Fetched 278 kB in 1s (375 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 155062 files and directories currently installed.)
Preparing to unpack .../zstd_1.3.3+dfsg-2ubuntu1.2_amd64.deb ...
Unpacking zstd (1.3.3+dfsg-2ubuntu1.2) ...
Setting up zstd (1.3.3+dfsg-2ubuntu1.2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
--2021-10-31 17:04:47--  https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.ta

Processing ./mesh-transformer-jax
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 590 kB 5.4 MB/s 
     |████████████████████████████████| 454.3 MB 14 kB/s 
  Created wheel for mesh-transformer: filename=mesh_transformer-0.0.0-py3-none-any.whl size=26330 sha256=e66c14562494309291018e1ad8dc9a4ecb59b91493c4d0dd32f8b47a866c0501
  Stored in directory: /root/.cache/pip/wheels/56/bd/89/b1f6b2f3d6b938d0c5812ee97756a1afd32521bea293543863
  Created wheel for jax: filename=jax-0.2.12-py3-none-any.whl size=682487 sha256=12ebc82c21dec690bbc8347150c73123044eaa92e3e77e03ba2850f6bcb9601c
  Stored in directory: /root/.ca

## Setup Model


In [ ]:
import os
import requests 
from jax.config import config

colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
requests.post(url)

# The following is required to use TPU Driver as JAX's backend.
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

Sometimes the next step errors for some reason, just run it again ¯\\\_(ツ)\_/¯

In [ ]:
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers

from mesh_transformer.checkpoint import read_ckpt_lowmem
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [ ]:
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

Here we create the network and load the parameters from the downloaded files. Expect this to take around 5 minutes.

In [ ]:
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt_lowmem(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:412: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


key shape (8, 2)
in shape (1, 2048)
dp 1
mp 8
Total parameters: 6053381344
read from disk/gcs in 225.196s


## Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when changed).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [ ]:
# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
def infer(context, top_p=0.9, temp=1.0, gen_len=512):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    print(f"completion done in {time.time() - start:06}s")
    return samples

print(infer("EleutherAI is")[0])

completion done in 57.803194761276245s
EleutherAI is one of the few exceptionally intelligent and knowledgeable people on the internet. She's is very knowledgeable in the practice of Etymology, and very adept at explaining it in very understandable language. As of May 2, 2010, she is a 48 year old artist, mother, and wife living in the DC area. She has a BA in English from Dickinson College and is an avid writer of both fiction and nonfiction.

Electra is a woman, in her early 20's, living in New York. She’s a recent college grad and works for a company in the Internet Marketing division. Her goal in life is to spread happiness and good deeds to others. She is extremely interested in teaching her children English.

Her husband who is a chef owns a restaurant in Nyack and works 60 hours a week. Electra is generally upbeat and always positive.

TagusMaximus is married, with two young sons. In her free time, he likes playing video games, reading, drawing, riding bikes, and occasionally ea

In [ ]:
#@title **Generate a new Song Title and Band Name** { form-width: "500px" }
top_p = 0.7 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}

context = """Create a new song title a new band name. Be creative!
###
Band name: The Execs
Song title: Company Meeting
###
Band name: The One Chords
Song title: Moving Down to Burlington
###
Band name:"""

print(infer(top_p=top_p, temp=temp, gen_len=512, context=context)[0])

completion done in 13.522092819213867s
Create a new song title a new band name. Be creative!
###
Band name: The Execs
Song title: Company Meeting
###
Band name: The One Chords
Song title: Moving Down to Burlington
###
Band name: The Spinning Circles
Song title: Scandal
###
Band name: The Mouth Breathers
Song title: Fight to Survive
###
Band name: Crying Cribs
Song title: Can't Get Used to Losing You
###
Band name: Hotline Bling
Song title: Free Fire
###
Band name: The Rock Band
Song title: Up the Revolution
###
Band name: A Scratch Band
Song title: Good for You
###
Band name: It's the Weekend
Song title: Live From the Bus to the Bars
###
Band name: Poor Superstars
Song title: Tales from the Psych Ward
###
Band name: Windy Rain
Song title: From What We Came From
###
Band name: Down for the Donut
Song title: The First Date
###
Band name: No Future in Sight
Song title: Tacky
###
Band name: Nothing Up My Sleeve
Song title: We Were One
###
Band name: The Steens
Song title: Written in Red In